In [1]:
import numpy as np
from functools import partial
import sobol_seq

**Problem 1 Pak Kun**

In [2]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

dim = 2
boundaries = np.array([(-10,10) for _ in range (dim)])
pop_size=100
max_gen=1000
F_init=0.5
CR_init=0.5
num_l=20
theta=1e-3
tau_d=0.4
s_max=100
print_gen=True
Hm = 50


**Problem 2 Pak Kun**

In [3]:
# def objective_function(x):
#     f1 = 0.5*np.sin(x[0]*x[1])-0.25*x[1]/np.pi - 0.5*x[0]
#     f2 = (1-0.25/np.pi)*(np.exp(2*x[0])-np.exp(1))+np.exp(1)*x[1]/np.pi - 2*np.exp(1)*x[0]
#     return np.array([f1,f2])

# boundaries = np.array([[-1,3],[-17,4]])
# dim = 2
# pop_size=1000
# max_gen=200
# F_init=0.5
# CR_init=0.5
# num_l=20
# theta=1e-3
# tau_d=0.36
# s_max=12
# print_gen=True
# Hm = 50

**Problem 1 Mastorakis**

In [4]:
# def objective_function(x):
#     f1 = x[0]**2+x[0]*x[1]-6
#     f2 = x[0]**2+x[1]**3+2*x[0]*x[1]**2-3
#     return np.array([f1,f2])

# dim = 2
# boundaries = np.array([(-10,10) for _ in range (dim)])
# pop_size=1000
# max_gen=200
# F_init=0.5
# CR_init=0.5
# num_l=20
# theta=1e-3
# tau_d=0.4
# s_max=20
# print_gen=True
# Hm = 50

# Objective Function

In [5]:
def root_objective_function(x:np.ndarray):
    res = 0
    F_array = objective_function(x)
    for f in F_array:
        res +=(f)**2
    return res

# def root_objective_function(x:np.ndarray):
#     res = 0
#     F_array = objective_function(x)
#     for f in F_array:
#         res +=abs(f)
#     return res

# def root_objective_function(x:np.ndarray):
#     F_array = objective_function(x)
#     denom = 0
#     for f in F_array:
#         denom += np.abs(f)
#     res = 1/(1+denom)
#     return -res

In [6]:
"""Characteristic Function"""
def chi_p(delta_j, rho):
    return 1 if delta_j <= rho else 0

"""Repulsion Function"""
def repulsion_function(x,
                       archive,
                       objective_func=root_objective_function,
                       beta=100,
                       rho=1e-8):
    f_x = objective_func(x)
    Rx = 0
    for x_star in archive:
        delta_j = np.linalg.norm(x-x_star)
        Rx += np.exp(-delta_j) * chi_p(delta_j, rho)
    Rx *= beta
    Rx += f_x
    return Rx

"""Fitness Function"""
def fitness_function(x,
                     archive,
                     objective_func=root_objective_function,
                     repulsion_func=repulsion_function):
    f_x = objective_func(x)
    if archive == []:
        return f_x
    else:
        return repulsion_func(x,archive)
    
# # Test the Functiuons
# arch = [np.array([1,2]),np.array([3,4]),np.array([-6.43,0.15])]
# x = np.array([-6.437160,0.155348])
# print(fitness_function(x,
#                        arch,
#                        objective_func=root_objective_function,
#                        repulsion_func=partial(repulsion_function,
#                                               objective_func = root_objective_function,
#                                               beta=1000,rho=0.01)))

In [7]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim: int,
                    npoint:int,
                    low=-10,
                    high=10,
                    sobol = True):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    if sobol == True:
        # Generate Sobol sequence points
        sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)
        # Scale the Sobol points to fit within the specified boundaries
        scaled_points = []
        for i in range(dim):
            a, b = boundaries[i]
            scaled_dim = a + sobol_points[:, i] * (b - a)
            scaled_points.append(scaled_dim)
        # Transpose the scaled points to get points per dimension
        scaled_points = np.array(list(map(list, zip(*scaled_points))))
    
    else:
        scaled_points = np.zeros((npoint, dim))
        for i in range(dim):
            min_val, max_val = boundaries[i]
            scaled_points[:, i] = np.random.uniform(min_val, max_val, npoint)

    return scaled_points

# def initialize_population(pop_size, dimensions, bounds):
#     population = np.random.rand(pop_size, dimensions)
#     lower_bounds, upper_bounds = np.asarray(bounds).T
#     diff = np.fabs(lower_bounds - upper_bounds)
#     return lower_bounds + population * diff

In [8]:
# Mutation function for DE
def mutate(population, F):
    # Vectorized mutation operation
    indices = np.arange(population.shape[0])
    np.random.shuffle(indices)
    r1, r2, r3 = population[indices[:3]]
    mutant = r1 + F * (r2 - r3)
    return mutant

def crossover(target, mutant, CR):
    cross_points = np.random.random(target.shape) < CR
    # Ensure at least one true crossover point
    if not np.any(cross_points):
        cross_points[np.random.randint(0, target.shape[0])] = True
    trial = np.where(cross_points, mutant, target)
    return trial

def mutation_penalty(x_i, subpop_i, boundaries, scaling_factor):
    """GENERATE: three distinct individuals xr1, xr1, xr1 from the current population randomly"""
    subpop_i_copy = subpop_i.copy()
    pop_ids = np.arange(subpop_i_copy.shape[0])
    indices_to_delete = np.where(np.all(subpop_i_copy == x_i, axis=1))[0] # Ensure that x_i is excluded from the selected subpopulation
    subpop_ids_no_i = np.delete(pop_ids, indices_to_delete, axis=0)
    subpop_i_copy = subpop_i_copy[subpop_ids_no_i]

    """MUTATION: Form the donor/mutation vector"""
    dv_i = mutate(subpop_i_copy,scaling_factor)

    """Set penalty for every donor vector that violates the boundaries"""
    for j in range (len(dv_i)):
        if dv_i[j] < boundaries[j,0]:
            dv_i[j] = (x_i[j]+boundaries[j,0])/2
        elif dv_i[j] > boundaries[j,1]:
            dv_i[j] = (x_i[j]+boundaries[j,1])/2
    return dv_i


In [9]:
"""FInd the closest points in a set to an initial point"""
def closest_solution(initial_point,set_of_points):
    diff = set_of_points-initial_point
    distances = np.linalg.norm(diff,axis=1)
    id_min_dist = np.argmin(distances)
    return id_min_dist,set_of_points[id_min_dist]

# # test the function
# alpha = np.array([1, 1])
# B = np.array([[5, 2],
#               [3, 4],
#               [5, 5]])
# closest_solution(alpha,B)

In [10]:
"""Calculate Euclidean distances and select t closest individuals"""
def subpopulating(individual, 
                  population, 
                  t,
                  return_index = False,
                  show_distances = False): 
    """Input"""
    # individual
    # population
    # t: max number of units in a subpopulation

    """Algorithm"""
    # Calculate the Euclidean distances from the individual to all others in the population
    distances = np.sqrt(np.sum((population - individual) ** 2, axis=1))
    # Get the indices of the individuals with the smallest distances
    closest_indices = np.argsort(distances)[:t]
    # Form the subpopulation with the closest individuals
    subpop = population[closest_indices]

    if show_distances == True:
        print(f'Distance: \n{distances[:t]}')
    if return_index == True:
        if t == 1:
            return closest_indices,subpop.flatten()
        else:
            return closest_indices,subpop
    else:
        if t == 1:
            return subpop.flatten()
        else:
            return subpop

# # Test the function
# # Assuming P is a numpy array of individuals where each individual is a point in n-dimensional space
# np.random.seed(0)
# P = np.random.rand(100, 2)  # Example: 100 individuals in a 2-dimensional space
# # print(f"P:{P}")
# print("")

# # The number of individuals to select with the smallest Euclidean distances
# t = 2
# # Forming subpopulations for each individual in P
# subpopulations = [subpopulating(xi, P, t) for xi in P]
# # print(f"subpopulations:{subpopulations}")
# print("")

# # Now subpopulations is a list of numpy arrays, each containing the t closest individuals to each xi in P (including xi itself)
# # For example, to access the subpopulation for the first individual in P:
# subpopulation_first_individual = subpopulations[0]
# print(f"subpopulation_first_individual:{subpopulation_first_individual}")

# [id1,id2],[v1,v2] = subpopulating(P[0],P,t,show_distances=True,return_index=True)
# v1,v2

In [11]:
def update_archive(x: np.ndarray,
                   objective_function,
                   archive,
                   theta,
                   tau_d,
                   s_max):
    """Input"""
    # x : Individual
    # theta : accuracy level
    # tau_d : distance radius
    # s_max : maximum archive size
    # archive : archive
    # s : archive current size

    f_x = objective_function(x)
    s = len(archive) # archive current size
    if f_x < theta: # x is a root
        # print(f'f({x})= {f_x}')
        if s == 0: # archive is empty
            archive.append(x)
            s+=1
        else:
            """Find the closest solution x_prime ∈ archive to x in the decision space"""
            dist_min = np.linalg.norm(x-archive[0])
            idx_min = 0
            x_prime= archive[idx_min]
            for i in range(1,len(archive)): 
                dist = np.linalg.norm(x-archive[i])
                if dist < dist_min:
                    dist_min = dist
                    x_prime = archive[i]
                    idx_min = i
            f_x_prime = root_objective_function(x_prime)
            if dist_min < tau_d: # x and x_prime are too close
                if f_x < f_x_prime:
                    x_prime = x
                    archive[idx_min] = x_prime
            else:
                if s < s_max:
                    archive.append(x)
                    s += 1
                else:       # archive is full
                    if f_x<f_x_prime:
                        x_prime = x
                        archive[idx_min] = x_prime
    return archive


# # Test the function
# x = np.array([-6.437160,0.155348]) # Individual
# theta = 1e-4 # accuracy level
# tau_d = 1e-1 # distance radius
# s_max = 3 # maximum archive size
# archive = [np.array([0,0]),np.array([1,2]),np.array([-6.4,0])] # archive
# update_archive(x,root_objective_function,archive,theta,tau_d,s_max)

In [12]:
def update_parameter(M_F,
                     M_CR,
                     Hm:int):
    """Input"""
    # MF: Historical memories of scaling factor of DE as F
    # MCR:Historical memories crossover rate of DE as CR
    # Hm: Size of Historical Memories

    # Randomly select an index
    hi = np.random.randint(0, Hm)
    # Generate Fi using the Cauchy distribution with the location parameter MF[hi] and scale 0.1
    Fi = np.random.standard_cauchy() * 0.1 + M_F[hi]
    # Generate CRi using the Gaussian distribution with mean MCR[hi] and standard deviation 0.1
    CRi = np.random.normal(M_CR[hi], 0.1)
    # Ensure CRi is within the range [0, 1] and Fi is within the range [0,1]
    Fi = np.clip(Fi, 0, 1)
    CRi = np.clip(CRi, 0, 1)
    return Fi, CRi

# # Coba
# MF = [0.5, 0.6, 0.7, 0.8, 0.9] 
# MCR = [0.1, 0.2, 0.3, 0.4, 0.5]
# update_parameter(MF,MCR,len(MF))

In [13]:
def meanWL(elements, weights):
    """
    Calculate the weighted Lehmer mean of elements.
    Lehmer mean is calculated as the weighted sum of the squares
    divided by the weighted sum of the elements.
    """
    numerator = np.sum(np.multiply(np.square(elements), weights))
    denominator = np.sum(np.multiply(elements, weights))
    return numerator / denominator if denominator != 0 else 0

# Define the weighted arithmetic mean function
def meanWA(elements, weights):
    """
    Calculate the weighted arithmetic mean of elements.
    This is the standard weighted mean.
    """
    return np.average(elements, weights=weights)

def update_history(M_F,M_CR,S_F,S_CR,k):
    weights = np.array([1 for _ in range (len(S_F))])
    if len(S_F)!=0:
        M_F[k] = meanWL(S_F,weights) 
    if len(S_CR)!=0:
        M_CR[k] = meanWA(S_CR,weights)
    return M_F,M_CR


# RADE Evaluation

Tanpa Memories

In [14]:
# np.random.seed(2)

# # Main RADE algorithm loop
# def RADE(archive, 
#          objective_func,
#          repulsion_func,
#          bounds, 
#          population_size, 
#          max_generation, 
#          F, 
#          CR,
#          num_l,
#          theta,
#          tau_d,
#          s_max,
#          print_gen = False):
#     dimensions = len(bounds)
#     population = generate_points(dim=dimensions,
#                                  npoint=population_size,
#                                  low=bounds[:,0],
#                                  high=bounds[:,1],
#                                  sobol=True)
#     fitness = np.asarray([objective_func(ind) for ind in population])
#     best_idx = np.argmin(fitness)
#     best = population[best_idx]
#     subpopA = np.array([subpopulating(xi, population, num_l) for xi in population])
#     for gen in range(max_generation):
#         for i in range(population_size):
#             x_i = population[i]
#             mutant = mutation_penalty(x_i,subpopA[i],bounds,F)
#             trial = crossover(population[i], mutant, CR)
#             trial_fitness = repulsion_func(trial)
            
#             if trial_fitness < fitness[i]:
#                 fitness[i] = trial_fitness
#                 population[i] = trial
#                 archive = update_archive(x = trial,
#                                         objective_function=objective_func,
#                                         archive=archive,
#                                         theta=theta,
#                                         tau_d=tau_d,
#                                         s_max=s_max)
#                 if trial_fitness < fitness[best_idx]:
#                     best_idx = i
#                     best = trial

#         if print_gen == True:
#             print(f"=========Generation {gen}=========")
#             # print(f"Best Fitness: {fitness[best_idx]}")
#             print(f'Archive:{archive}')
#     return best, fitness[best_idx], archiveA



# archiveA = []
# best_solution, best_fitness, archiveA = RADE(objective_func=root_objective_function, 
#                                             repulsion_func=partial(repulsion_function, 
#                                                                     archive=archiveA,
#                                                                     objective_func=root_objective_function),
#                                             archive=archiveA,
#                                             bounds = boundaries, 
#                                             population_size=250, 
#                                             max_generation=1000, 
#                                             F=0.9, 
#                                             CR=0.8,
#                                             num_l=20,
#                                             theta=1e-3,
#                                             tau_d=0.4,
#                                             s_max=100,
#                                             print_gen=True)
# print("Best Solution:", best_solution)
# print("Best Fitness:", best_fitness)
# print(f'Roots: {archiveA}')

Dengan Memories

In [15]:
np.random.seed(0)

# Main RADE algorithm loop
def RADE(archive, 
         objective_func,
         bounds, 
         population_size, 
         max_generation, 
         memories_F, 
         memories_CR,
         num_l,
         theta,
         tau_d,
         archive_size_max,
         print_gen = False):
    
    dimensions = len(bounds)
    population = generate_points(dim=dimensions,
                                 npoint=population_size,
                                 low=bounds[:,0],
                                 high=bounds[:,1],
                                 sobol=True)
    fitness = np.asarray([objective_func(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best = population[best_idx]
    subpopA = np.array([subpopulating(xi, population, num_l) for xi in population])
    Hm = len(memories_F)
    k=0

    for gen in range(max_generation):
        S_F, S_CR = [],[]
        for i in range(population_size):
            F_i,CR_i = update_parameter(memories_F,memories_CR,Hm)
            x_i = population[i]
            mutant = mutation_penalty(x_i,subpopA[i],bounds,F_i)
            trial = crossover(x_i, mutant, CR_i)
            trial_fitness = fitness_function(trial, 
                                             archive,
                                             objective_func=objective_func,
                                             repulsion_func=repulsion_function)
            id_closest_trial,closest_trial = closest_solution(trial,population)
            closest_trial_fitness = fitness_function(closest_trial, 
                                                     archive,
                                                     objective_func=objective_func,
                                                     repulsion_func=repulsion_function)
            if trial_fitness < closest_trial_fitness:
                # print(trial_fitness, fitness[i])
                fitness[id_closest_trial] = trial_fitness
                population[id_closest_trial] = trial
                archive = update_archive(x = trial,
                                        objective_function=objective_func,
                                        archive=archive,
                                        theta=theta,
                                        tau_d=tau_d,
                                        s_max=archive_size_max)
                S_F.append(F_i)
                S_CR.append(CR_i)
                if trial_fitness < fitness[best_idx]:
                    best_idx = i
                    best = trial

        if print_gen == True:
            print(f"=========Generation {gen}=========")
            # print(f"Best Fitness: {fitness[best_idx]}")
            print(f'Archive:{archive}')
        
            # print(f'S_F: {S_F}\nS_CR: {S_CR}')
        """Update parameter history"""
        if (len(S_F)!=0) & (len(S_CR)!=0):
            memories_F,memories_CR = update_history(memories_F,memories_CR,S_F,S_CR,k)
            # print(f'M_F: {M_F}\nM_CR: {M_CR}')
            k +=1
            if k >= Hm:
                k = 1
    return best, fitness[best_idx], archive


M_CR = np.ones(Hm)*CR_init
M_F = np.ones(Hm)*F_init
archiveA = []
best_solution, best_fitness, archiveA = RADE(objective_func=root_objective_function, 
                                            archive=archiveA,
                                            bounds = boundaries, 
                                            population_size=pop_size, 
                                            max_generation=max_gen, 
                                            memories_F=M_F, 
                                            memories_CR=M_CR,
                                            num_l=num_l,
                                            theta=theta,
                                            tau_d=tau_d,
                                            archive_size_max=s_max,
                                            print_gen=True)
print("Best Solution:", best_solution)
print("Best Fitness:", best_fitness)
print(f'Roots: {archiveA}')


=========Generation 0=========
Archive:[]
=========Generation 1=========
Archive:[]
=========Generation 2=========
Archive:[]
=========Generation 3=========
Archive:[]
=========Generation 4=========
Archive:[]
=========Generation 5=========
Archive:[]
=========Generation 6=========
Archive:[]
=========Generation 7=========
Archive:[]
=========Generation 8=========
Archive:[]
=========Generation 9=========
Archive:[]
=========Generation 10=========
Archive:[]
=========Generation 11=========
Archive:[]
=========Generation 12=========
Archive:[]
=========Generation 13=========
Archive:[]
=========Generation 14=========
Archive:[]
=========Generation 15=========
Archive:[]
=========Generation 16=========
Archive:[]
=========Generation 17=========
Archive:[]
=========Generation 18=========
Archive:[]
=========Generation 19=========
Archive:[]
=========Generation 20=========
Archive:[]
=========Generation 21=========
Archive:[]
=========Generation 22=========
Archive:[]
=========Generation 2

=========Generation 25=========
Archive:[]
=========Generation 26=========
Archive:[]
=========Generation 27=========
Archive:[]
=========Generation 28=========
Archive:[]
=========Generation 29=========
Archive:[]
=========Generation 30=========
Archive:[]
=========Generation 31=========
Archive:[]
=========Generation 32=========
Archive:[]
=========Generation 33=========
Archive:[]
=========Generation 34=========
Archive:[]
=========Generation 35=========
Archive:[]
=========Generation 36=========
Archive:[]
=========Generation 37=========
Archive:[]
=========Generation 38=========
Archive:[]
=========Generation 39=========
Archive:[]
=========Generation 40=========
Archive:[]
=========Generation 41=========
Archive:[]
=========Generation 42=========
Archive:[]
=========Generation 43=========
Archive:[]
=========Generation 44=========
Archive:[]
=========Generation 45=========
Archive:[]
=========Generation 46=========
Archive:[]
=========Generation 47=========
Archive:[]
=========Ge

In [16]:
print(len(archiveA))
mak = 100
for i in range (len(archiveA)):
    for j in range (i,len(archiveA)):
        if (i!=j) and (mak > np.linalg.norm(archiveA[i]-archiveA[j])):
            mak = np.linalg.norm(archiveA[i]-archiveA[j])
            print(mak)
            print(archiveA[i], archiveA[j])
mak

6
6.8283938992233875
[0.66357378 0.68077034] [-6.11229552 -0.1645453 ]
1.644987778399347
[0.66357378 0.68077034] [-0.9375      1.05832504]
0.45892996790487195
[-6.11229552 -0.1645453 ] [-6.44193664  0.1547562 ]
0.44260752907495365
[-0.15625     6.44550557] [0.16458134 6.14059792]


0.44260752907495365

Automatic

In [17]:
# import rdeal as rd
# import importlib
# import time
# importlib.reload(rd)

# start = time.time()

# M_CR = np.ones(Hm)*CR_init
# M_F = np.ones(Hm)*F_init
# archiveA = []
# best_solution, best_fitness, archiveA = rd.RADE(objective_func=root_objective_function, 
#                                             archive=archiveA,
#                                             bounds = boundaries, 
#                                             population_size=pop_size, 
#                                             max_generation=max_gen, 
#                                             memories_F=M_F, 
#                                             memories_CR=M_CR,
#                                             num_l=num_l,
#                                             theta=theta,
#                                             tau_d=tau_d,
#                                             archive_size_max=s_max,
#                                             print_gen=True)

# end = time.time()
# print("Best Solution:", best_solution)
# print("Best Fitness:", best_fitness)
# print(f'Roots: {archiveA}')

# print(f"Execution time: {end - start} seconds.")

# Visualization

In [18]:
# import matplotlib.pyplot as plt

# if dim == 2:
#     archiveA = np.array(archiveA)
#     # Create a scatter plot
#     plt.scatter(archiveA[:,0], archiveA[:,1])

#     # Add grid, labels and title for context
#     plt.grid(True)
#     plt.xlabel('X-axis')
#     plt.ylabel('Y-axis')
#     plt.title('2D Cartesian Plot of Points')

#     # Show the plot
#     plt.show()


In [19]:
# import numpy as np
# A = np.array([[-0.260599 ,0.622531],
# [1.530510 ,-10.202200],
# [1.663420 ,-16.282800],
# [1.654580 ,-15.819200], 
# [1.433950 ,-6.820760],
# [1.578220 ,-12.176700], 
# [1.337430 ,-4.140440], 
# [0.500000 ,3.141590],
# [0.299449 ,2.836930],
# [1.604570 ,-13.362900],
# [1.294360 ,-3.137220],
# [1.481320 ,-8.383610]])
# mak = 100
# for i in range (len(A)):
#     for j in range (i,len(A)):
#         if (i!=j) and (mak > np.linalg.norm(A[i]-A[j])):
#             mak = np.linalg.norm(A[i]-A[j])
#             print(mak)
#             print(A[i], A[j])
# mak